# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著完成三種不同特徵類型的三種資料操作, 觀察結果
- 思考一下, 這三種特徵類型, 哪一種應該最複雜/最難處理

# [作業重點]
- 完成剩餘的八種 類型 x 操作組合 (In[6]~In[13], Out[6]~Out[13])
- 思考何種特徵類型, 應該最複雜

In [2]:
# 載入基本套件
import pandas as pd
import numpy as np

# 讀取訓練與測試資料
data_path = 'Data/D7/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')
df_train.shape

(891, 12)

In [3]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
# 重組資料成為訓練 / 預測用格式
train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1) # 只留下解釋變數
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test]) # 將資料合併，我們先專注於解釋變數的資料分析上
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
df.dtypes.reset_index()

,index,0
0,Pclass,int64
1,Name,object
2,Sex,object
3,Age,float64
4,SibSp,int64
5,Parch,int64
6,Ticket,object
7,Fare,float64
8,Cabin,object
9,Embarked,object


In [25]:
# 秀出資料欄位的類型與數量
dtype_df = df.dtypes.reset_index() #df.dtpyes 可以 list 出每個 column 的 data type，其資料型態為 df.series 
# 再透過 reset_index() 來將資料轉成一 data frame，但其實 reset index 本質上是將 df 的 index 從新設定 (從 0 開始)
dtype_df.columns = ["Count", "Column Type"]
dtype_df = dtype_df.groupby("Column Type").aggregate('count').reset_index()
dtype_df

,Column Type,Count
0,int64,3
1,float64,2
2,object,5


In [28]:
#確定只有 int64, float64, object 三種類型後, 分別將欄位名稱存於三個 list 中
int_features = []
float_features = []
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64':
        float_features.append(feature)
    elif dtype == 'int64':
        int_features.append(feature)
    else:
        object_features.append(feature)
print(f'{len(int_features)} Integer Features : {int_features}\n')
print(f'{len(float_features)} Float Features : {float_features}\n')
print(f'{len(object_features)} Object Features : {object_features}')

3 Integer Features : ['Pclass', 'SibSp', 'Parch']

2 Float Features : ['Age', 'Fare']

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


# 作業1 
* 試著執行作業程式，觀察三種類型 (int / float / object) 的欄位分別進行( 平均 mean / 最大值 Max / 相異值 nunique )  
中的九次操作會有那些問題? 並試著解釋那些發生Error的程式區塊的原因?  

# 作業2
* 思考一下，試著舉出今天五種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類?  
所以三大類特徵中，哪一大類處理起來應該最複雜?

In [29]:
# 例 : 整數 (int) 特徵取平均 (mean)
df[int_features].mean()

Pclass    2.294882
SibSp     0.498854
Parch     0.385027
dtype: float64

In [38]:
# 請依序列出 三種特徵類型 (int / float / object) x 三種方法 (平均 mean / 最大值 Max / 相異值數量 nunique) 的其餘操作
print('-'*5,'Mean','-'*5)
print(df[int_features].mean())
print('-'*5,'Max','-'*5)
print(df[int_features].max())
print('-'*5,'Nunique','-'*5)
print(df[int_features].nunique())

----- Mean -----
Pclass    2.294882
SibSp     0.498854
Parch     0.385027
dtype: float64
----- Max -----
Pclass    3
SibSp     8
Parch     9
dtype: int64
----- Nunique -----
Pclass    3
SibSp     7
Parch     8
dtype: int64


In [40]:
# 請依序列出 三種特徵類型 (int / float / object) x 三種方法 (平均 mean / 最大值 Max / 相異值數量 nunique) 的其餘操作
print('-'*5,'Mean','-'*5)
print(df[float_features].mean())
print('-'*5,'Max','-'*5)
print(df[float_features].max())
print('-'*5,'Nunique','-'*5)
print(df[float_features].nunique())

----- Mean -----
Age     29.881138
Fare    33.295479
dtype: float64
----- Max -----
Age      80.0000
Fare    512.3292
dtype: float64
----- Nunique -----
Age      98
Fare    281
dtype: int64


In [43]:
# 請依序列出 三種特徵類型 (int / float / object) x 三種方法 (平均 mean / 最大值 Max / 相異值數量 nunique) 的其餘操作
print('-'*5,'Mean','-'*5)
print(df[object_features].mean())
print('-'*5,'Max','-'*5)
print(df[object_features].max())
print('-'*5,'Nunique','-'*5)
print(df[object_features].nunique())

----- Mean -----
Series([], dtype: float64)
----- Max -----
Name      van Melkebeke, Mr. Philemon
Sex                              male
Ticket                      WE/P 5735
dtype: object
----- Nunique -----
Name        1307
Sex            2
Ticket       929
Cabin        186
Embarked       3
dtype: int64


C:\Users\User\AppData\Local\Temp\ipykernel_16900\2169446333.py:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(df[object_features].mean())
C:\Users\User\AppData\Local\Temp\ipykernel_16900\2169446333.py:5: FutureWarning: The default value of numeric_only in DataFrame.max is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(df[object_features].max())


## Q1:

Error 出現在當我們想對 object type 的 column 進行 average 跟 find maximum 時，原因其實很直觀，因為 oject type 還沒轉換成數字之前可以視為是字串，而單純的字串(類別型資料)沒有辦法進行加減乘除，也沒有大小差異(順序型資料則有大小，例如第一名 > 第二名 > 第三名)，因此才會跳出 error。

此外，當我們想要對 float type 進行 nunique (計算相異數值的數量) 時，其實是沒有太大的意義的，原因在於 float 的資料通常會有數以百計的數值，因此我們對 float type 做 counting 的時候其實意義並不大，一般來說，我們的處理手法會是定義一個 interval 並且將資料切分成 N 塊 block 然後再計算 block 的 counting， 藉此來去觀察該資料的分佈狀況。

## Q2:

課程提及的五大資料型態 : 數值 / 類別 / 二元 / 次序 / 時間
除了上面提及的資料型態之外，其實還有一種資料型態很常見 : 區間資料 (interval) 
區間資料跟數值資料 (比例尺度資料) 之間最大的不同在於比例尺度的資料有絕對的 0 (表示"無"或是"沒有"的概念)
但區間資料的 0 並不是無或是沒有的意思。
舉例溫度就是一種區間資料，因為 0 度並非表示沒有溫度 !
一般來說，區間資料的處理上 (建模) 大多都會被視為是一般的數值資料，但其實處理手法可以更加細膩，
最重要的特點是區間資料常常具有上下限，因此我們可以加入 truncate 或是 censor 的方法來處理。